In [1]:
# 1.定义langchain模型调用工具函数

from dotenv import load_dotenv
from langchain_core.tools import tool
from typing import Callable
from pydantic import BaseModel, Field
load_dotenv()

# 定义一个函数类型别名
OperationFunction = Callable[[float, float], float]
'''
# 建议添加参数约束（需要修改工具定义）
parameters={
    "type": "object",
    "properties": {
        "operation": {
            "type": "string",
            "enum": ["add", "subtract", "multiply", "divide", "power"]  # 限定可选值
        },
        "a": {"type": "number"},
        "b": {"type": "number"}
    },
    "required": ["operation", "a", "b"]
}
'''
class CalculateInput(BaseModel):
    """严格约束的计算参数格式"""
    operation: str = Field(..., enum=["add", "subtract", "multiply", "divide", "power"], description="必须使用英文操作名：add/subtract/multiply/divide/power")
    a: float = Field(..., description="第一个操作数")
    b: float = Field(..., description="第二个操作数")

@tool(args_schema=CalculateInput)
def calculate(operation: str, a: float, b: float) -> float:
    """执行数学四则运算和幂运算。支持的操作类型：add（加）, subtract（减）, multiply（乘）, divide（除）, power（幂）"""
    # 定义支持的运算和相应的函数
    operations: dict[str, OperationFunction] = {
        "add": lambda x, y: x + y,
        "subtract": lambda x, y: x - y,
        "multiply": lambda x, y: x * y,
        "divide": lambda x, y: x / y if y != 0 else float('inf'),  # 防止除以零
        "power": lambda x, y: x ** y
    }
    
    print(f"Executing {operation} operation with {a} and {b}")
    
    # 检查运算是否支持
    if operation in operations:
        return operations[operation](a, b)
    else:
        raise ValueError("Unsupported operation")



In [2]:
# 查天气工具函数
import os
import requests
class WeatherInput(BaseModel):
    city: str = Field(..., description="要查询的城市名称/id，城市名称如：温州、上海、北京，需要utf8 urlencode")

@tool(args_schema=WeatherInput)
def get_current_weather(city: str, unit: str = "metric") -> str:
    """查询指定城市的天气情况"""
    try:
        url = "http://apis.juhe.cn/simpleWeather/query"
        params = {
            "city": city,
            "key": os.getenv("JUHE_WEATHER_API_KEY")
        }
        response = requests.get(url, params=params)
        print(city, os.getenv("JUHE_WEATHER_API_KEY"))
        # 解析响应结果
        if response.status_code == 200:
            data = response.json()
            realtime = data['result']['realtime']
            return f"""
            {city}当前天气：
            - 温度：{realtime['temperature']}{'°C' if unit=='metric' else '°F'}
            - 湿度：{realtime['humidity']}%
            - 天气状况：{realtime['info']}{realtime['direct']}{realtime['power']}
            """
        else:
            # 网络异常等因素，解析结果异常。可依据业务逻辑自行处理。
            return f"天气查询失败,请求异常"
    except Exception as e:
        return f"天气查询失败：{str(e)}"


In [3]:
# 2.初始化大模型并绑定工具函数
import os
from langchain_ollama import ChatOllama

# 只有ChatOllama有绑定工具函数
llm = ChatOllama(
    base_url=os.environ['OLLAMA_BASE_URL'],  # 可配置为内部服务器地址
    model="qwq:32b",
    temperature=0.3
)
llm_with_tools = llm.bind_tools(tools=[calculate, get_current_weather])  # 可绑定多个工具

In [6]:
# 3. 调用模型并处理函数调用
from langchain_core.messages import HumanMessage
from typing import Union

def process_query(query: str) -> Union[float, str]:
    # 发送用户问题
    response = llm_with_tools.invoke(query)
    
    # 检查是否触发工具调用
    if not response.tool_calls:
        return response.content

    # 执行工具调用
    for tool_call in response.tool_calls:
        if tool_call["name"] == "calculate":
            args = tool_call["args"]
            print(f"正在调用工具：{tool_call['name']}，参数：{args}")
            try:
                # 执行计算并返回结果
                result = calculate.invoke(args)
                return f"运算结果：{result}"
            except Exception as e:
                return f"计算错误：{str(e)}"
        elif tool_call["name"] == "get_current_weather":
            args = tool_call["args"]
            print(f"正在调用工具：{tool_call['name']}，参数：{args}")
            try:
                # 执行天气查询并返回结果
                result = get_current_weather.invoke(args)
                return f"当前天气：{result}"
            except Exception as e:
                return f"天气查询错误：{str(e)}"
        else:
            return "未找到匹配的工具"
    
    return response.content

# 4. 测试调用
print(process_query("查询一下乐昌的天气如何？")) 

正在调用工具：get_current_weather，参数：{'city': '乐昌'}
乐昌 43e7adc29309a02aeb4ce150f98e7f28
当前天气：
            乐昌当前天气：
            - 温度：15°C
            - 湿度：54%
            - 天气状况：晴西北风2级
            
